In [2]:
base_dir = "G:/My Drive/2020 ORGANISATION/1. PROJECTS/qualitative analysis of literature/110 CITATION ANALYSIS/000 data/sociology wos"

In [3]:
name_blacklist = [
    "*us", 'Press', 'Knopf', '(January', 'Co', 'London', 'Bros', 'Books', 'Wilson','[anonymous]'
]

In [4]:
from knowknow import *
from counter import wos_counter

In [5]:
prelim_count = Dataset("sociology-wos-all")

loading variable sociology-wos-all/groups from disk
loading variable sociology-wos-all/_attributes from disk


In [6]:
filtered = Dataset("sociology-wos-74a")
filtered.groupings = [] # clear... not yet

loading variable sociology-wos-74a/groups from disk
loading variable sociology-wos-74a/group_reps from disk
loading variable sociology-wos-74a/_attributes from disk


In [7]:
final = Dataset('sociology-wos-74b')

loading variable sociology-wos-74b/groups from disk
loading variable sociology-wos-74b/_attributes from disk


# Counting over the entire dataset

In [7]:
if not prelim_count.exists():
    cnt = wos_counter(
        wos_txt_base = base_dir,
        output_database = prelim_count.name,
        RUN_EVERYTHING = False,
        name_blacklist=name_blacklist
    )
    
    cnt.run()

In [8]:
prelim_count.set_attribute('RELIABLE_DATA_STARTS_HERE', 1960)
prelim_count.set_attribute('RELIABLE_DATA_ENDS_HERE', 2015)
prelim_count.set_attribute('type','wos')

In [4]:
max( prelim_count.items('fy') )

loading variable sociology-wos-all/ind ___ fy from disk


2020

# Examine journals to determine what to drop

In [7]:
zero_sofars = []
# how I decide whether to keep a journal
def filt(j):
    
    try:
        tt = prelim_count.trend('fj', j)
    except invalid_entry:
        return False

    # get the years for which there is a positive count of BIBLIOGRAPHIES
    ys = set(y for y,c in tt._c.items() if c > 0)

    # this is a weird error, probably requires me to rerun (rework for efficiency) my counting algos
    # might even be reduced once I actually filter the journals
    if not len(ys):
        #raise Exception("no years for journal %s" % j)
        return False

    # it must have been around for at least 10 years, with their first year being sometime before 1995
    if not(
        (max(ys)-min(ys) >= 10 and min(ys) <= 1995 and max(ys) >= 2015)
    ): return False

    # 5 years consecutively, then drop
    ever_nonzero = False
    zero_sofar = 0
    zero_sofar_M = 0
    for y in range(tt.data_start, tt.data_end):
        c = tt._c[ y ]
        
        if c > 0:
            ever_nonzero = True

        if not ever_nonzero:
            continue

        if c == 0:
            zero_sofar += 1
        else:
            zero_sofar_M = max(zero_sofar_M, zero_sofar)
            zero_sofar = 0

        if zero_sofar >= 5:
            return False
        
    # at least 100 papers published total
    if not(
        prelim_count(fj=j).docs >= 100
    ): return False
    
    zero_sofars.append(zero_sofar_M)
    if zero_sofar_M > 0:
        print(j,zero_sofar_M)

    return True

In [8]:
journals = list(prelim_count.items('fj'))

print( "There are %s journals total in `%s`" % (len(journals), prelim_count.name) )

keep, drop = set(), set()
for j in journals:
    if filt(j):
        keep.add(j)
    else:
        drop.add(j)
        
print( "Only %s meet my standards" % (len(keep) ) )

loading variable sociology-wos-all/ind ___ fj from disk
There are 312 journals total in `sociology-wos-all`
loading variable sociology-wos-all/ind ___ fj.fy from disk
loading variable sociology-wos-all/ind ___ fy from disk
loading variable sociology-wos-all/doc ___ fj from disk
acta sociologica 1
american journal of economics and sociology 1
american journal of sociology 1
american sociological review 1
journal of leisure research 1
journal of marriage and family 1
journal of the history of sexuality 1
law & society review 3
politics & society 3
review of religious research 1
social forces 1
social problems 1
social science quarterly 1
sociologia 1
sociologia ruralis 3
sociological inquiry 1
sociological quarterly 1
sociological review 1
sociology of education 1
sociology of sport journal 1
sociology-the journal of the british sociological association 1
Only 73 meet my standards


In [9]:
Counter(zero_sofars)

Counter({1: 18, 0: 52, 3: 3})

## journals to keep

In [10]:
print(", ".join(sorted(keep)))

acta sociologica, american journal of economics and sociology, american journal of sociology, american sociological review, annals of tourism research, anthrozoos, archives europeennes de sociologie, armed forces & society, british journal of sociology, british journal of sociology of education, canadian journal of sociology-cahiers canadiens de sociologie, comparative studies in society and history, contributions to indian sociology, deviant behavior, discourse & society, economy and society, ethnic and racial studies, european sociological review, gender & society, human ecology, human studies, international journal of intercultural relations, international sociology, journal for the scientific study of religion, journal of contemporary ethnography, journal of health and social behavior, journal of historical sociology, journal of law and society, journal of leisure research, journal of marriage and family, journal of mathematical sociology, journal of the history of sexuality, langu

## journals to drop

In [11]:
print(", ".join(sorted(drop)))

african sociological review, afro-american studies, ager-revista de estudios sobre despoblacion y desarrollo rural, agriculture and human values, american journal of cultural sociology, american sociologist, american west, anduli, annals of phenomenological sociology, annual review of sociology, applied social studies, archiv fur sozialwissenschaft und sozialpolitik, australian and new zealand journal of sociology, australian outlook, berliner journal fur soziologie, biodemography and social biology, bms-bulletin of sociological methodology-bulletin de methodologie sociologique, body & society, cahiers internationaux de sociologie, cambio-rivista sulle trasformazioni sociali, canadian graduate journal of sociology and criminology, canadian review of sociology and anthropology, canadian review of sociology and anthropology-revue canadienne de sociologie et d anthropologie, canadian review of sociology-revue canadienne de sociologie, center for settlement studies. occasional papers. univ

# compute counts over just these selected journals

In [37]:
if not filtered.exists() or True:
    # clear the groups, this shouldn't happen yet.
    cnt = wos_counter(
        wos_txt_base = base_dir,
        output_database = filtered.name,
        RUN_EVERYTHING = False,
        name_blacklist=name_blacklist,
        journals_filter=keep
    )
    
    cnt.run()

File 0/386: 74501-75000.txt
Document: 0
Citations: 0
File 50/386: 3501-4000.txt
Document: 10276
Citations: 305043
File 100/386: 28501-29000.txt
Document: 19502
Citations: 575931
File 150/386: 53501-54000.txt
Document: 28876
Citations: 783578
File 200/386: 4501-5000.txt
Document: 39815
Citations: 984788
File 250/386: 29501-30000.txt
Document: 57003
Citations: 1262270
File 300/386: 54501-55000.txt
Document: 75537
Citations: 1521073
File 350/386: 79501-80000.txt
Document: 88298
Citations: 1652774
loading variable sociology-wos-74a/groups from disk
loading variable sociology-wos-74a/group_reps from disk
loading variable sociology-wos-74a/groups from disk
loading variable sociology-wos-74a/group_reps from disk
loading variable sociology-wos-74a/groups from disk
loading variable sociology-wos-74a/group_reps from disk
loading variable sociology-wos-74a/groups from disk
loading variable sociology-wos-74a/group_reps from disk
loading variable sociology-wos-74a/groups from disk
loading variable 

In [38]:
filtered.set_attribute('RELIABLE_DATA_STARTS_HERE', 1960)
filtered.set_attribute('RELIABLE_DATA_ENDS_HERE', 2015)
filtered.set_attribute('type','wos')

# now compute summary statistics about citations.

There are many we will want to exclude for almost all analyses, in order to keep more cooccurrence counts

In [15]:
if True:
    t = ttDF(
        dataset_name = filtered.name,
        dtype = 'c',
        debug=False
    ).run()

loading variable sociology-wos-74a/groups from disk
loading variable sociology-wos-74a/group_reps from disk
loading variable sociology-wos-74a/ind ___ c from disk
Item 0/1705081 ('Abbasi, I.|dawlat i pahlavi va)
Current memory usage is 344.665238MB; Peak was 866.211967MB
defaultdict(<class 'int'>, {})
loading variable sociology-wos-74a/doc ___ c from disk
loading variable sociology-wos-74a/ind ___ c.fy from disk
loading variable sociology-wos-74a/ind ___ fy from disk
Item 50000/1705081 (Apple, M.|2010|educ researcher,v39,p152)
Current memory usage is 1339.450226MB; Peak was 1815.714224MB
defaultdict(<class 'int'>, {'at least one citation': 50000, 'less than 5 citations... dropped.2': 46577, 'not enough parts': 16, 'less than 5 citations... dropped.': 769, 'passed tests pre-blacklist': 2561, 'invalid entry': 77})
Item 100000/1705081 (Baslevent, C.|2014|soc indic res,v118,p33)
Current memory usage is 1357.022018MB; Peak was 1815.714224MB
defaultdict(<class 'int'>, {'at least one citation

In [16]:
len( filtered.load_variable('c.ysum') )

loading variable sociology-wos-74a/c.ysum from disk


90796

# and group those which make the cut

In [17]:
# note that unfortunately this currently requires YSUM
#   and thus will be based on the filters embedded in that...
#   AND these decisions will propagate further, because counters only consider if they're in the groups
# some decisions need to be made here...
if True:
    from grouper import Grouper
    g = Grouper(dataset=filtered)
    g.run()

loading variable sociology-wos-74a/c.ysum from disk
90796 strings total...
sample articles: ['Aalen, O.|1978|ann stat,v6,p701', 'Aaronson, D.|1998|j hum resour,v33,p915', 'Aaronson, D.|2000|j urban econ,v47,p356', 'Aaronson, N.|1993|j natl cancer i,v85,p365', 'Aarts, O.|2008|rev relig res,v50,p16', 'Aas, C.|2005|ann tourism res,v32,p28', 'Aassve, A.|2002|j eur soc policy,v12,p259', 'Aassve, A.|2003|demography,v40,p105', 'Aassve, A.|2007|eur j popul,v23,p315', 'Abberley, P.|1987|disability handicap,v2,p5']
sample books: ['Aaker, D.|managing brand equit', 'Aapola, S.|young femininity gir', 'Aaron, H.|painful prescription', 'Aaron, H.|politics professors', 'Aaron, H.|studies ec income ma', 'Aaron, H.|who pays property ta', 'Abbas, T.|muslim britain commu', 'Abbey, A.|res quality life', 'Abbey, E.|monkey wrench gang', 'Abbot, A.|what is case explori']
49930 articles, 39407 books to group
8694 books have some connection to others in a group
34500 groups total
[(32736, 23), (16710, 19), (880

# recount on this focused group, grouping citations

In [12]:
#toload = ['fj','fy','ty','c.fy','c.fj','fj.fy','c','fy.ty','fj.ty','ta','fy.ta','fj.ta','c.fj.fy','ffa','ffa.fy','ffa.fj','c.ffa','fa','fa.fy','fa.fj','c.fa']

if not final.exists() or True:
    cnt = wos_counter(
        wos_txt_base = base_dir,
        output_database = final.name,
        RUN_EVERYTHING = True,
        name_blacklist=name_blacklist,
        journals_filter= keep,
        groups= filtered.load_variable('groups'),
        group_reps= filtered.load_variable('group_reps'),
        #debug=True
    )
    
    if True:
        cnt.run()
    else:
        for k in toload:
            cnt.doc[k] = final.by(*k.split(".")).docs
            cnt.ind[k] = final.by(*k.split(".")).cits
            
        cnt.count_ages()
    
    final.set_attribute('RELIABLE_DATA_STARTS_HERE', 1960)
    final.set_attribute('RELIABLE_DATA_ENDS_HERE', 2015)
    final.set_attribute('type','wos')

loading variable sociology-wos-74a/groups from disk
loading variable sociology-wos-74a/group_reps from disk
File 0/386: 74501-75000.txt
Document: 0
Citations: 0
File 50/386: 3501-4000.txt
Document: 10276
Citations: 53549
File 100/386: 28501-29000.txt
Document: 19502
Citations: 60119
File 150/386: 53501-54000.txt
Document: 28876
Citations: 64835
File 200/386: 4501-5000.txt
Document: 39815
Citations: 69550
File 250/386: 29501-30000.txt
Document: 57003
Citations: 77116
File 300/386: 54501-55000.txt
Document: 75537
Citations: 82486
File 350/386: 79501-80000.txt
Document: 88298
Citations: 83837
File 0/386: 74501-75000.txt
Document: 0
Citations: 83858
File 50/386: 3501-4000.txt
Document: 10276
Citations: 83858
File 100/386: 28501-29000.txt
Document: 19502
Citations: 83858
File 150/386: 53501-54000.txt
Document: 28876
Citations: 83858
File 200/386: 4501-5000.txt
Document: 39815
Citations: 83858
File 250/386: 29501-30000.txt
Document: 57003
Citations: 83858
File 300/386: 54501-55000.txt
Docume

# do the simple string grouper, finally...

In [9]:
grp = require('wos-counter/grouper/SimpleStringGrouper')
grp(final).run()

loading variable sociology-wos-74b/ind ___ c from disk


In [12]:
final(c='Andrews, F.|social indicators we').cits

366

In [13]:
t = final.trend('c', 'Andrews, F.|social indicators we')

loading variable sociology-wos-74b/ind ___ c.fy from disk
loading variable sociology-wos-74b/ind ___ fy from disk


In [14]:
t.first

1976